In [2]:
import pandas as pd

In [14]:
df = pd.read_csv("../data/full_sentiment_dataset_cleaned.csv")
df.head()

,date,ticker,positive,neutral,negative,return_1d,return_2d,return_3d,ticker_encoded
0,2017-01-31,21CF,0.000167,0.999619,0.000214,-0.000319,-0.002868,-0.002868,0
1,2017-02-01,21CF,0.000000,0.000000,0.000000,NaN,NaN,NaN,0
2,2017-02-02,21CF,0.000000,0.000000,0.000000,NaN,NaN,NaN,0
3,2017-02-03,21CF,0.000000,0.000000,0.000000,NaN,NaN,NaN,0
4,2017-02-04,21CF,0.000000,0.000000,0.000000,NaN,NaN,NaN,0


In [4]:
import numpy as np

In [19]:
window = 3
seq = []

for _, group in df.groupby('ticker'):
    group = group.reset_index(drop=True)
    for i in range(len(group)):
        end = min(i + window, len(group))
        sentiment = group.loc[i:end - 1, ['positive', 'neutral', 'negative']].values 
        
        if len(sentiment) < window:
            pad = window - len(sentiment)
            fill = np.zeros((pad, 3))
            sentiment = np.vstack((sentiment, fill))
            
        ticker_id = group.loc[i, 'ticker_encoded']
        ticker_feature = np.full((window, 1), ticker_id)
        features = np.hstack([sentiment, ticker_feature])
            
        if end < len(group):
            target = group.loc[end, ['return_1d', 'return_2d', 'return_3d']].values
            
            if not np.any(pd.isnull(target)) and not np.all(features == 0) and not np.all(target == 0):
                seq.append((features, target))

In [20]:
X = np.array([x[0] for x in seq], dtype = np.float32)
y = np.array([x[1] for x in seq], dtype = np.float32)

In [21]:
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader

In [22]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=42)